In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader ,random_split
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler
import copy

C:\Users\yoavl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sequence_length = 10000

def create_torch_dataset(size):
    if not size:
        size = 561
    all_data = []
    scaler = MinMaxScaler()
    columns = ['Input Voltage', 'Output Voltage', 'Tachometer']
    # num_rows = 122824
    num_rows = sequence_length
    for i in range(1, size):
        temp_df = pd.read_csv(f'../PHM09_competition_1/Run_{i}.csv', names=columns, nrows=num_rows)
        normalized_data = scaler.fit_transform(temp_df)
        temp_tensor = torch.tensor(normalized_data, dtype=torch.float32)
        # temp_tensor = temp_tensor.unsqueeze(dim=0)  # Adding a channel dimension
        all_data.append(temp_tensor)
    
    all_data_tensor = torch.stack(all_data, dim=0)
    return all_data_tensor

dataset_tensor = create_torch_dataset(size=50)

In [3]:
dataset_tensor.shape

torch.Size([49, 10000, 3])

In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [5]:
def train_model(model, train_dataset, val_dataset, n_epochs):
    best_loss = float('inf')
    # best_model_wts = copy.deepcopy(model.state_dict())
    history = {'train': [], 'val': []}
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    train_loss = 0

    for epoch in range(1, n_epochs + 1):
        model = model.train()
        train_loss = []
        for seq_true in train_dataset:
            optimizer.zero_grad()
            seq_pred = model(seq_true)
            loss = loss_fn(seq_pred, seq_true)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        # Eval loop 
        val_loss = []
        model = model.eval()
        with torch.no_grad():
            for seq_true in val_dataset:
                seq_pred = model(seq_true)
                loss = loss_fn(seq_pred,seq_true)
                val_loss.append(loss.item())
            
        avg_train_loss = sum(train_loss) / len(train_loss)
        avg_val_loss = sum(val_loss) / len(val_loss)
        history['train'].append(avg_train_loss)
        history['val'].append(avg_val_loss)

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            # best_model_wts = copy.deepcopy(model.state_dict())

        print(f'Epoch {epoch}: train loss {avg_train_loss} val loss {avg_val_loss}')
        # model.load_state_dict(best_model_wts)
    
    return model, history
    
# Prepare DataLoaders
dataset_tensor = dataset_tensor.view(560, 1, -1)  # Reshape tensor
train_size = int(0.8 * len(dataset_tensor))
val_size = len(dataset_tensor) - train_size
train_dataset, val_dataset = random_split(dataset_tensor, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)

model = LSTMModel(2625, 64, 2)  # Assuming hidden_layer_size is 64 and output_size is 2
model, history = train_model(model ,train_dataset, val_dataset, n_epochs=3)

C:\Users\yoavl\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 2625])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2) must match the size of tensor b (2625) at non-singleton dimension 1